In [1]:
#%% ---------------------------------Libraries---------------------------------
from __future__ import division
from __future__ import print_function
# import torch
import textdistance as td
import pdb
import glob
import sys
import random
import numpy as np
import matplotlib.pyplot as plt

import time
import os
import copy as cp
from collections import deque
import pandas as pd
from scipy.optimize import curve_fit

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from tqdm import tqdm_notebook as tqdm
%matplotlib inline

In [32]:
class dgwo():
    def __init__(self, pool, indv_size, pop_size, target='', mutation_rate = 0.1, crossover_rate = 0.5):
        self.pool = pool
        self.indv_size = indv_size
        self.pop_size = pop_size
        self.target = target
        self.indv = []
        self.pop = []
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate

    def individual(self,):
        self.indv = random.sample(self.pool,self.indv_size)
        return self.indv 

    def populate(self,):
        for n in range(self.pop_size):
            self.pop.append(self.individual())
        return self.pop

    def objf1(self,):
        '''
        This function calculate if the given individual equal to the designated target.
        sum(indiv) = target
        target - sum(indiv) = 0
        '''
        return abs(self.target - sum(self.indv))

    def objf2(self,):
        '''
        This function check if the given two text is the same or not using the levenshtein normalized distance,
        if the to text is identical, return 0.0 else return 0.0> = number <= 1.0. 
        '''
        return td.levenshtein.normalized_distance(''.join(self.indv), self.target)

    def mutate(self,):
        '''
        this function return the mutation for chiled by mutation rate as a factor.
        '''
        mutate_value = int(round(self.mutation_rate * len(self.indv))
        mutation_pool = random.sample(self.pool, mutate_value)
        for i in range(mutate_value):
            mutation_value = random.choice(mutation_pool)
            mutation_pool.pop(mutation_value)
            self.indiv[random.randint(0,self.iniv_size)] = mutation_value
        return self.indiv

SyntaxError: invalid syntax (<ipython-input-32-189f2e96e80b>, line 41)

In [210]:
def crossover2(indv1, indv2, crossover_rate):
    '''
    this function return the cross over between two agents defined by crossover rate.
    '''
    indv1, indv2 = np.array(indv1), np.array(indv2)
    crossover_value = int(round(crossover_rate * len(indv1),0))
    
    flag = np.random.randint(0,2)
    if flag==1:
        child = np.hstack((indv1[:crossover_value] , indv2[crossover_value:]))
        
    else:
        child = np.hstack((indv2[:crossover_value] , indv1[crossover_value:]))
    return child

In [211]:
indv1

array([1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 1])

In [229]:
indv1 = pop[1]
indv2 = pop[2]
crossover_rate = 0.5
crossover2(indv1, indv2, crossover_rate)

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 0])

In [238]:
def crossover3(indv1, indv2, indv3, crossover_rate):
    '''
    this function return the cross over between three indvs defined by crossover rate.
    '''
    indv1, indv2, indv3 = np.array(indv1), np.array(indv2), np.array(indv3)
    crossover_value = int(round(crossover_rate * len(indv1),0))
    flag = np.random.randint(0,3)
    if flag == 0: 
        child = np.hstack((indv1[:crossover_value] , 
                           indv2[crossover_value:crossover_value*2], 
                           indv3[crossover_value*2:]))
    elif flag == 1: 
        child = np.hstack((indv1[:crossover_value] , 
                           indv3[crossover_value:crossover_value*2], 
                           indv2[crossover_value*2:]))
    else: 
        child = np.hstack((indv3[:crossover_value] , 
                           indv2[crossover_value:crossover_value*2], 
                           indv1[crossover_value*2:]))
    return child

In [241]:
indv1 = pop[1]
indv2 = pop[2]
indv3 = pop[3]
crossover_rate = 0.3
crossover3(indv1, indv2, indv3, crossover_rate)

array([0, 0, 0, 1, 0, 1, 0, 0, 0, 0])

In [300]:
gen_no=1*20; run_no = 5;POP_SIZE = 25;mutate_rate = 0.2

In [301]:
columns = ['run_no','loss','Excec_Time','best_solution', 'best_10_solutions']
df = pd.DataFrame(columns = columns)
conv_df = pd.DataFrame(np.zeros((gen_no)))

for run_no_i in range(run_no):

    Convergence_curve = []                    
                       
    Last_Alpha_score =float("inf")

    IND_SIZE = X_train.shape[1]
    #-------------------------------------------------

#     pdb.set_trace()
    FILE_PATH = 'data/best_features_scores_all.csv'
    init_arr = init_pop(FILE_PATH)
    Positions = populate(POP_SIZE, init_arr=init_arr)

    pos_fit = []

    fitness_arr = np.array([[1, np.asarray(Positions[0])]])
    #--------------------------------------------------
    timerStart = time.time()
    df_list = []
    #--------------------------------------------------    
    for gen_no_i in tqdm(range(gen_no)):
#         sys.stdout.write("\r%.2f%%" % ((gen_no_i+1)*100/gen_no))
#         sys.stdout.flush()
        for agent in  Positions:
            idx = np.where(agent == 1)[0].tolist()
            new_X_train = X_train[:, idx]
            new_X_test = X_test[:, idx]
            fitness = objf_lgbm(new_X_train, new_X_test, y_train, y_test)
            arr1 = np.array([[fitness, np.asarray(agent)]])
            fitness_arr=np.vstack((fitness_arr,arr1))  
            pos_fit.append([fitness, agent])

        fitness_arr = fitness_arr[fitness_arr[:,0].argsort()]    
        Alpha_score, Alpha_pos = fitness_arr[0][0], list(fitness_arr[0][1])
        Beta_score, Beta_pos   = fitness_arr[1][0], list(fitness_arr[1][1])
        Delta_score, Delta_pos = fitness_arr[2][0], list(fitness_arr[2][1])
    #---------------------------------------------------------------------    
        if Alpha_score<Last_Alpha_score:
            Last_Alpha_score = Alpha_score
        Convergence_curve.append(Last_Alpha_score)
        
    #--------------------------Start prey search----------------------------------------------------    
    #       a=2-l*((2)/gen_no); # a decreases linearly fron 2 to 0 --> a =
        a= 2*(1-(gen_no_i/gen_no)**2)
        for i in range(POP_SIZE):               
            X1,X2,X3 = [], [], []
            r1=random.random() # if r1>0.5: A1=[0:2] -->converge --> crossover
            r2=random.random() # if r1<0.5: A1=[-2:0] --> Diverge --> Mutation
            A1=2*a*r1-a; # Equation (3.3)
            C1=2*r2; # Equation (3.4)
            B0 = []            
            agent_fitness = pos_fit[i][0]
            B0 = cp.deepcopy(pos_fit[i][1])
            D_alpha = abs((agent_fitness - C1*Alpha_score)/agent_fitness) 
            if abs(A1)>=1:
                #Exploration
                X1 = mutate(B0, D_alpha)
            if abs(A1)<1:
                #Exploitation
                X1 = crossover2(Alpha_pos, B0, D_alpha)

            r1=random.random() # if r1>0.5: A1=[0:2] -->converge --> crossover
            r2=random.random() # if r1<0.5: A1=[-2:0] --> Diverge --> Mutation
            A2=2*a*r1-a; # Equation (3.3)
            C2=2*r2; # Equation (3.4)

            B0 = []
            B0 = cp.deepcopy(pos_fit[i][1])
            D_beta = abs((agent_fitness - C2*Beta_score)/agent_fitness)
            if abs(A2)>=1:
                #Exploration
                X2 = mutate(B0, D_beta)
            if abs(A2)<1:
                #Exploitation
                X2 = crossover2(Beta_pos, B0, D_beta)

            r1=random.random() # if r1>0.5: A1=[0:2] -->converge --> crossover
            r2=random.random() # if r1<0.5: A1=[-2:0] --> Diverge --> Mutation
            A3=2*a*r1-a; # Equation (3.3)
            C3=2*r2; # Equation (3.4)

            B0 = []
            B0 = cp.deepcopy(pos_fit[i][1])
            D_delta = abs((agent_fitness - C3*Delta_score)/agent_fitness)
            if abs(A3)>=1:
                #Exploration
                X3 = mutate(B0, D_delta)
            if abs(A3)<1:
                #Exploitation
                X3 = crossover2(Delta_pos, B0, D_delta)

            Positions[i] = crossover3(X1, X2, X3, 0.3)

        pos_fit = []   
        # Remove All Rows from fitness_arr Except first three (Alpha_ps, Beta_Pos, Delta_pos)
        fitness_arr = np.delete(fitness_arr,np.s_[3:], axis=0)

    Positions = np.array(pd.DataFrame(Positions).drop_duplicates())
    fitness_arr = np.zeros((len(Positions),2), dtype='object')
    for i,agent in  enumerate(Positions):
        idx = np.where(agent == 1)[0].tolist()
        new_X_train = X_train[:, idx]
        new_X_test = X_test[:, idx]
        fitness = objf_lgbm(new_X_train, new_X_test, y_train, y_test)
        fitness_arr[i,0] = fitness
        fitness_arr[i,1] = agent.tolist()

    fitness_arr = fitness_arr[fitness_arr[:,0].argsort()] 
    df_list = fitness_arr.tolist()
    df = pd.DataFrame(df_list, columns=['loss', 'features'])
    df.to_csv('data/best_features_scores_{}.csv'.format(run_no), index=False)

/home/ai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


/home/ai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


/home/ai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


/home/ai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


/home/ai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


In [303]:
agent = np.array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1])

In [304]:
idx = np.where(agent == 1)[0].tolist()
new_X_train = X_train[:, idx]
new_X_test = X_test[:, idx]
fitness = objf_lgbm(new_X_train, new_X_test, y_train, y_test)

In [305]:
fitness

0.048688094076088206

In [306]:
idx

[2, 6, 12, 13, 15, 16, 17, 19, 21, 23, 24, 26, 28, 29, 30, 31]

In [309]:
SelectedFeatureInds = [0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 21, 22, 23, 24, 29, 30, 31]
idx=np.zeros(X.shape[1], dtype='bool')
idx[SelectedFeatureInds]=1
idx = idx.tolist()
new_X_train = X_train[:, idx]
new_X_test = X_test[:, idx]
fitness = objf_lgbm(new_X_train, new_X_test, y_train, y_test)

In [310]:
fitness

0.021404169281182495

In [311]:
new_X_train = X_train
new_X_test = X_test
fitness = objf_lgbm(new_X_train, new_X_test, y_train, y_test)
fitness

0.01949847777080571